In [1]:
import json, os
import numpy as np
from tqdm import tqdm_notebook

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [3]:
def load_circuits(data_dir):
    circuits = []
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.json')][2:3]
    for data_file in data_files:
        for entry in tqdm_notebook(json.load(open(data_file, 'r'))):
            circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
    return circuits

circuits = load_circuits('./data/mbd_datasets2/theta_0.05pi/train/')
print(len(circuits))

/var/folders/kj/mfqyg_tj77nf4gt67lvscdkw0000gn/T/ipykernel_76110/4227574046.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for entry in tqdm_notebook(json.load(open(data_file, 'r'))):


  0%|          | 0/500 [00:00<?, ?it/s]

500


In [12]:
def encode_data(circuits, properties):

    def recursive_dict_loop(my_dict, parent_key=None, out=[], target_key1=None, target_key2=None):
        for key, val in my_dict.items():
            if isinstance(val, dict):
                recursive_dict_loop(val, key, out, target_key1, target_key2)
            else:
                if parent_key and target_key1 in str(parent_key) and key == target_key2:
                    out += [val]
        return out

    vec = [np.mean(recursive_dict_loop(properties, target_key1='cx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='id', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='sx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='x', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='rz', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='', target_key2='readout_error'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='', target_key2='t1'))]
    vec += [np.mean(recursive_dict_loop(properties, target_key1='', target_key2='t2'))]
    vec = torch.tensor(vec) * 10000 # put it in the same order of magnitude as the number of gates

    bin_size = 0.1 * np.pi
    num_angle_bins = int(np.ceil(4 * np.pi / bin_size))
    X = torch.zeros([len(circuits), len(vec) + 3 + num_angle_bins], dtype=torch.float64)
    X[:, :len(vec)] = vec[None, :]

    for i, circ in enumerate(circuits):
        gate_counts = circ.count_ops()
        X[i, len(vec):len(vec)+3] = torch.tensor(
            [gate_counts['rz'],  gate_counts['sx'], gate_counts['cx']]
        )

    def count_gates_by_rotation_angle(circuit):
        angles = []
        for instr, qargs, cargs in circuit.data:
            if instr.name in ['rx', 'ry', 'rz'] and len(qargs) == 1:
                angles += [instr.params[0]]
        bin_edges = np.arange(-2 * np.pi, 2 * np.pi + bin_size, bin_size)
        counts, _ = np.histogram(angles, bins=bin_edges)
        bin_labels = [f"{left:.2f} to {right:.2f}" for left, right in zip(bin_edges[:-1], bin_edges[1:])]
        angle_bins = {label: count for label, count in zip(bin_labels, counts)}
        return list(angle_bins.values())

    for i, circ in enumerate(circuits):
        gate_counts = count_gates_by_rotation_angle(circ)
        X[i, len(vec)+3:] = torch.tensor(gate_counts)

    return X

X = encode_data(circuits, properties)
X[0]

tensor([111.3405,  69.8825,  51.4567,  51.4567,  45.1815,  73.5210,  67.5698,
         62.5218, 112.0000,  98.0000,  24.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   8.0000,
         14.0000,   0.0000,   8.0000,   0.0000,  21.0000,   1.0000,   0.0000,
          6.0000,   0.0000,   5.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          8.0000,  21.0000,   8.0000,   0.0000,   0.0000,  12.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000], dtype=torch.float64)

In [ ]:

my_dataset = TensorDataset(torch.Tensor(X), torch.Tensor(y))

my_dataloader = DataLoader(my_dataset) # create your dataloader

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

train_data_dir = 'data'  # the directory containing the JSON files
train_dataset = JSONDataset(train_data_dir)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_dir = 'data'  # the directory containing the JSON files
test_dataset = JSONDataset(test_data_dir)
test_dataloader = DataLoader(test_dataset, batch_size=100000, shuffle=False)

model = MLP(input_dim=100, hidden_dim=50, output_dim=10)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_dataloader), loss.item()))

# Evaluate your model on the test set
test_inputs = torch.stack([torch.Tensor(d) for d in test_data])
test_outputs = model(test_inputs)
test_loss =
